## https://python.langchain.com/docs/tutorials/local_rag/

In [1]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import OllamaLLM

In [2]:
# 기본 예제
llm = OllamaLLM(model="llama3.2:latest")


template = "{country}의 수도는?"

prompt = PromptTemplate.from_template(template=template)
chain = prompt | llm | StrOutputParser()

result = chain.invoke({"country", "한국"})
print(result)

부산


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# 텍스트 파일 경로 설정
file_path = "data/data.txt"

# 텍스트 파일을 읽어서 로딩
try:
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
except FileNotFoundError:
    raise Exception(f"Failed to load text file at path: {file_path}")

# 텍스트를 문서 형태로 변환 (단순히 리스트로 사용 가능)
document = Document(page_content=text)

# 텍스트를 분할하기 위해 텍스트 분할기 사용
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
all_splits = text_splitter.split_documents([document])

# 결과 출력
i = 1
for split in all_splits:
    print(f"Chunk {i}:{split.page_content}\n")
    i += 1


Chunk 1:앤제 포스테코글루 감독의 말과 행동은 달랐다.

토트넘은 10일 오후 11시(한국시간) 영국 런던에 위치한 토트넘 훗스퍼 스타디움에서 열린 2024-25시즌 프리미어리그(PL) 11라운드에서 입스위치 타운에 1-2로 패배했다. 이로써 토트넘은 공식전 2연패에 빠졌고, 리그 10위까지 추락했다. 입스위치는 리그 11경기 만에 첫 승을 얻었다.

경기 전 가장 화두에 오른 주제는 '손흥민 몸 상태 관리'였다. 토트넘은 '손흥민 부상'으로 고단한 한 달을 겪었다. 손흥민은 9월 말 유럽축구연맹(UEFA) 유로파리그(UEL) 카라바흐전에서 햄스트링 부상을 당했다. 이후 약 3주간 긴 재활을 거쳐 복귀했지만, 한 경기만에 또 다시 제외됐다. 결국 또 다시 회복기를 거쳐 지난 아스톤 빌라전에서 복귀했다. 포스테코글루 감독은 후반 11분 손흥민을 빼며 그의 몸 상태를 신중하게 관리했다.

Chunk 2:이후 포스테코글루 감독은 한국 국가대표팀과의 협력을 원했다. 포스테코글루 감독은 글로벌 매체 'ESPN'과의 인터뷰에서 "나는 국가대표팀 감독과 그들의 계획에 관여하는 것을 주저하는 편이다. 나 역시 호주 대표팀 감독이었을 때, 구단 감독들이 내게 어떻게 하라고 지시하는 것을 좋아하지 않았다"며 조심스러운 입장을 밝혔다.

이어 포스테코글루 감독은 "그러나 상호 협력적인 접근이 있을 것이라 생각하고 싶다. 손흥민이 구단과 대표팀에서 최근 어떠한 일정을 겪었는지 함께 살펴보고 있다고 생각한다. 우리는 신체적으로 건강하고 좋은 컨디션인 손흥민이 필요하다. 그래야만 손흥민이 기여할 수 있다. 그러한 맥락에서 나는 구단과 대표팀 모두 긍정적인 결과를 얻을 것이라 확신한다"고 덧붙였다.

Chunk 3:손흥민의 나이와 체력을 고려해, 양측 모두 신중하게 관리하자는 이야기였다. 그러나 포스테코글루 감독은 단 하루 만에 말과 행동이 다른 모습을 보였다. 손흥민은 입스위치전 선발 출전했다. 손흥민은 전반 초반부터 활발한 움직임으로 결정적인 득점 기회를 창출했다. 그러나 토트

In [4]:
# https://ollama.com/blog/embedding-models

from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

#local_embeddings = OllamaEmbeddings(model="nomic-embed-text") # 이것 사용했는데 잘안됨

## ollama pull mxbai-embed-large 다른것 설치함
local_embeddings = OllamaEmbeddings(model="mxbai-embed-large") 
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

embedded_query = local_embeddings.embed_query("LangChain 에 대해서 상세히 알려주세요.")
# 임베딩 차원 출력
len(embedded_query)

1024

In [5]:
question = "대한체육회장 선거 날짜는?"
#question = "부상을 당한 사람은 누구인가요?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [6]:
docs[0]

Document(metadata={}, page_content='카이클은 이 전화 인터뷰에서 사사키의 행선지에 대한 중요한 발언을 남겼다. 바로 사사키가 다저스로 간다는 것이었다. 카이클은 “사사키가 다저스와 이미 계약을 끝냈다는 소문을 들었다”고 전했다. 다만 어디까지나 ‘풍문’을 들었다는 것으로, 카이클은 곧바로 “다저스가 사사키를 위한 계획이 있다는 말을 들었다. 만약 내가 라스베이거스의 도박사라면 다저스에 한 표를 던지겠으나, 난 사실을 정확하게 알지 못한다”고 덧붙였다.\n\n사사키의 행선지로는 다저스를 포함해 뉴욕 양키스, 뉴욕 메츠 같은 ‘빅마켓’ 팀들이 주로 거론되고 있다. 샌디에이고 파드리스 역시 후보군에 속해 있다.\n\n\n사사키 로키. 게티이미지코리아')

In [7]:
docs[1]

Document(metadata={}, page_content='사사키 로키. 연합뉴스')

In [8]:
docs[2]

Document(metadata={}, page_content='손흥민의 나이와 체력을 고려해, 양측 모두 신중하게 관리하자는 이야기였다. 그러나 포스테코글루 감독은 단 하루 만에 말과 행동이 다른 모습을 보였다. 손흥민은 입스위치전 선발 출전했다. 손흥민은 전반 초반부터 활발한 움직임으로 결정적인 득점 기회를 창출했다. 그러나 토트넘은 전반 31분 선제골을 내주며 끌려갔다. 이후 전반 43분에도 라두 드라구신의 결정적인 실수로 두 번째 골까지 내줬다.\n\n포스테코글루 감독 입장에서는 후반 추격이 시급했다. 후반 23분 로드리고 벤탄쿠르의 추격골이 나왔다. 끝내 1-2로 추격의 불씨를 되살린 토트넘이었다. 포스테코글루 감독은 지난 빌라전과는 다른 선택을 했다. 손흥민을 빼지 않은 것. 계속해서 좋은 모습을 보여준 손흥민을 필두로 균형을 잡겠다는 의도였다. 결국 손흥민은 풀타임을 소화했지만, 토트넘은 1-2 충격패를 맞았다.')

In [9]:
docs[3]

Document(metadata={}, page_content='이기흥 회장은 직무 정지 상태에서도 내년 1월 14일에 열리는 제42대 대한체육회장 선거에 나설 수 있다.\n\n후보등록기간은 다음 달 24일부터 25일까지다.\n\n이기흥 회장이 12일 체육회 스포츠공정위원회의 3연임 최종심사를 통과해 차기 회장 선거에 출마한다면, 정관에 따라 직무 정지 상태로 입후보해야 한다.\n\n문체부의 이번 조처로 약 한 달 먼저 이 회장의 직무가 정지됐다.\n\n문체부 관계자는 \'이기흥 회장이 선거에 출마할 경우 어차피 직무 정지에 들어갈 예정이어서 이번 조처가 실효성이 떨어질 것 같다\'는 말에 "대한체육회장 선거와는 별개의 행정 조치"라며 "규정된 법률에 따라 행한 것이라고 보면 된다"고 거푸 강조했다.\n\n아울러 "대한체육회장 선거에 영향을 미치면 선거 개입이 된다"며 "이번 조처가 선거에 영향을 미칠 수도 없다"고 말했다.')

In [10]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2:latest",
)

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

#question = "토트넘은 몇위가 되었나요?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

'다음 달 24일부터 25일까지이다.'

In [12]:
from langchain.vectorstores import FAISS

In [13]:
vectorstore = FAISS.from_documents(embedding=local_embeddings , documents=all_splits)

In [14]:
vectorstore.docstore._dict

{'01fb81ad-bb42-4e0b-bcc7-a1e58ba6ce27': Document(metadata={}, page_content="앤제 포스테코글루 감독의 말과 행동은 달랐다.\n\n토트넘은 10일 오후 11시(한국시간) 영국 런던에 위치한 토트넘 훗스퍼 스타디움에서 열린 2024-25시즌 프리미어리그(PL) 11라운드에서 입스위치 타운에 1-2로 패배했다. 이로써 토트넘은 공식전 2연패에 빠졌고, 리그 10위까지 추락했다. 입스위치는 리그 11경기 만에 첫 승을 얻었다.\n\n경기 전 가장 화두에 오른 주제는 '손흥민 몸 상태 관리'였다. 토트넘은 '손흥민 부상'으로 고단한 한 달을 겪었다. 손흥민은 9월 말 유럽축구연맹(UEFA) 유로파리그(UEL) 카라바흐전에서 햄스트링 부상을 당했다. 이후 약 3주간 긴 재활을 거쳐 복귀했지만, 한 경기만에 또 다시 제외됐다. 결국 또 다시 회복기를 거쳐 지난 아스톤 빌라전에서 복귀했다. 포스테코글루 감독은 후반 11분 손흥민을 빼며 그의 몸 상태를 신중하게 관리했다."),
 'c0f77afb-824f-4833-959f-901804d75e5c': Document(metadata={}, page_content='이후 포스테코글루 감독은 한국 국가대표팀과의 협력을 원했다. 포스테코글루 감독은 글로벌 매체 \'ESPN\'과의 인터뷰에서 "나는 국가대표팀 감독과 그들의 계획에 관여하는 것을 주저하는 편이다. 나 역시 호주 대표팀 감독이었을 때, 구단 감독들이 내게 어떻게 하라고 지시하는 것을 좋아하지 않았다"며 조심스러운 입장을 밝혔다.\n\n이어 포스테코글루 감독은 "그러나 상호 협력적인 접근이 있을 것이라 생각하고 싶다. 손흥민이 구단과 대표팀에서 최근 어떠한 일정을 겪었는지 함께 살펴보고 있다고 생각한다. 우리는 신체적으로 건강하고 좋은 컨디션인 손흥민이 필요하다. 그래야만 손흥민이 기여할 수 있다. 그러한 맥락에서 나는 구단과 대표팀 모두 긍정적인 결과를 얻을 것이라 확신한다"고 덧붙였다.'),
 '325ebab

In [15]:
docs = vectorstore.similarity_search(question)
len(docs)

4

In [16]:
for doc in docs:
    print(doc)
    print("----")

page_content='카이클은 이 전화 인터뷰에서 사사키의 행선지에 대한 중요한 발언을 남겼다. 바로 사사키가 다저스로 간다는 것이었다. 카이클은 “사사키가 다저스와 이미 계약을 끝냈다는 소문을 들었다”고 전했다. 다만 어디까지나 ‘풍문’을 들었다는 것으로, 카이클은 곧바로 “다저스가 사사키를 위한 계획이 있다는 말을 들었다. 만약 내가 라스베이거스의 도박사라면 다저스에 한 표를 던지겠으나, 난 사실을 정확하게 알지 못한다”고 덧붙였다.

사사키의 행선지로는 다저스를 포함해 뉴욕 양키스, 뉴욕 메츠 같은 ‘빅마켓’ 팀들이 주로 거론되고 있다. 샌디에이고 파드리스 역시 후보군에 속해 있다.


사사키 로키. 게티이미지코리아'
----
page_content='사사키 로키. 연합뉴스'
----
page_content='손흥민의 나이와 체력을 고려해, 양측 모두 신중하게 관리하자는 이야기였다. 그러나 포스테코글루 감독은 단 하루 만에 말과 행동이 다른 모습을 보였다. 손흥민은 입스위치전 선발 출전했다. 손흥민은 전반 초반부터 활발한 움직임으로 결정적인 득점 기회를 창출했다. 그러나 토트넘은 전반 31분 선제골을 내주며 끌려갔다. 이후 전반 43분에도 라두 드라구신의 결정적인 실수로 두 번째 골까지 내줬다.

포스테코글루 감독 입장에서는 후반 추격이 시급했다. 후반 23분 로드리고 벤탄쿠르의 추격골이 나왔다. 끝내 1-2로 추격의 불씨를 되살린 토트넘이었다. 포스테코글루 감독은 지난 빌라전과는 다른 선택을 했다. 손흥민을 빼지 않은 것. 계속해서 좋은 모습을 보여준 손흥민을 필두로 균형을 잡겠다는 의도였다. 결국 손흥민은 풀타임을 소화했지만, 토트넘은 1-2 충격패를 맞았다.'
----
page_content='이기흥 회장은 직무 정지 상태에서도 내년 1월 14일에 열리는 제42대 대한체육회장 선거에 나설 수 있다.

후보등록기간은 다음 달 24일부터 25일까지다.

이기흥 회장이 12일 체육회 스포츠공정위원회의 3연임 최종심사를 통과해 차기 회장 선거에 출마한다